In [38]:
import pandas as pd
import numpy as np

In [39]:
cfpb = pd.read_csv('../1_data_raw/cfpb_loan_level_LI.csv')
cfpb.head()

C:\Users\Fefe\AppData\Local\Temp\ipykernel_24592\3054842870.py:1: DtypeWarning: Columns (23,24,25,27,28,29,30,31,32,33,34,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  cfpb = pd.read_csv('../1_data_raw/cfpb_loan_level_LI.csv')


,Unnamed: 0,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,507,2022,RVDPPPGHCGZ40J4VQ731,35004,NY,36103.0,3.610320e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,...,NaN,NaN,NaN,5369,45.41,146400,74.0,1189,2845,42
1,519,2022,RVDPPPGHCGZ40J4VQ731,35004,NY,36103.0,3.610317e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,...,NaN,NaN,NaN,7731,15.77,146400,135.0,2873,3498,41
2,521,2022,RVDPPPGHCGZ40J4VQ731,35004,NY,36059.0,3.605940e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,...,NaN,NaN,NaN,4363,92.51,146400,94.0,1084,1275,64
3,531,2022,RVDPPPGHCGZ40J4VQ731,35004,NY,36059.0,3.605942e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,...,NaN,NaN,NaN,5948,24.61,146400,109.0,1768,1943,67
4,534,2022,RVDPPPGHCGZ40J4VQ731,35004,NY,36103.0,3.610315e+10,C,VA:First Lien,Single Family (1-4 Units):Site-Built,...,NaN,NaN,NaN,4543,85.74,146400,70.0,667,1019,53


In [40]:
cfpb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115871 entries, 0 to 115870
Data columns (total 100 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Unnamed: 0                                115871 non-null  int64  
 1   activity_year                             115871 non-null  int64  
 2   lei                                       115871 non-null  object 
 3   derived_msa_md                            115871 non-null  int64  
 4   state_code                                115869 non-null  object 
 5   county_code                               115871 non-null  float64
 6   census_tract                              115857 non-null  float64
 7   conforming_loan_limit                     115697 non-null  object 
 8   derived_loan_product_type                 115871 non-null  object 
 9   derived_dwelling_category                 115871 non-null  object 
 10  derived_ethnicity  

Notes:  
- Some categories look like they are partially one-hot-encoded already, but not completely (Not boolean values? need to check if binary)  
- A lot of personal information about applicants  
- A lot of columns with null values (will need to decide if drop, impute etc.)

### Explanation

- action_taken:
    - 1: Loan originated
    - 2: Application approved but not accepted
    - 3: Application denied by financial institution
    - 4: Application withdrawn by applicant
    - 5: File closed for incompleteness
    - 6: Loan purchased by the institution
    - 7: Preapproval request denied by financial institution
    - 8: Preapproval request approved but not accepted

- purchaser_type
    - 0: Loan was not originated or was not sold in calendar year covered by register
    - 1: Fannie Mae
    - 2: Ginnie Mae
    - 3: Freddie Mac
    - 4: Farmer Mac
    - 5: Private securitization
    - 6: Commercial bank, savings bank or savings association
    - 7 1: credit union, mortgage bank, or finance company
    - 7 2: Life insurance company
    - 8: Affiliate institution
    - 9: Other type of purchaser

- preapproval
    - 1: Preapproval was requested
    - 2: Preapproval was not requested

- loan_type | derived_loan_product_type is the string version
    - 1: Conventional (any loan other than FHA, VA, FSA, or RHS loans)
    - 2: FHA-insured (Federal Housing Administration)
    - 3: VA-guaranteed (Veterans Administration)
    - 4: FSA/RHS (Farm Service Agency or Rural Housing Service)

## Y-Values

- action_taken
- loan_amount
- property_value

### Plans

I believe this dataset can lend itself to a classification or clustering.  

Filter the following:  

- loan_purpose: 1 (purchase)
- business_or_commercial_purpose: 2 (no, personal purpose)
- occupancy_type: 1 (owner occupied)
- total_units: 1 (1 unit)

Drop:  

- Drop all _2 _3 _4 _5 columns (too many null values)
- Drop all columns with more than 50% null values
- Manually drop the following:
    - Column 0 (ID)
    - derived_msa_md
    - State_code
    - Census_tract
    - conforming_loan_limit
    - loan_purpose
    - lien_status
    - reverse_mortgage
    - open_end_line_of_credit
    - business_or_commercial_purpose
    - ... more mentioned in array below





In [41]:

cfpb.query('loan_purpose == 1 and business_or_commercial_purpose == 2', inplace=True)

cfpb.drop([col for col in cfpb.columns if '_2' in col or '_3' in col or '_4' in col or '_5' in col], axis=1, inplace=True)
# Drop columns with more than 50% missing values
cfpb.dropna(thresh=len(cfpb)/2, axis=1, inplace=True)
manual_drop = ['derived_msa_md', 'state_code', 'census_tract', 'conforming_loan_limit', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open_end_line_of_credit', 'business_or_commercial_purpose', 'rate_spread', 'hoepa_status', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'construction_method', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest']
cfpb.drop(manual_drop, axis=1, inplace=True)

cfpb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47252 entries, 1 to 115870
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         47252 non-null  int64  
 1   activity_year                      47252 non-null  int64  
 2   lei                                47252 non-null  object 
 3   county_code                        47252 non-null  float64
 4   derived_loan_product_type          47252 non-null  object 
 5   derived_dwelling_category          47252 non-null  object 
 6   derived_ethnicity                  47252 non-null  object 
 7   derived_race                       47252 non-null  object 
 8   derived_sex                        47252 non-null  object 
 9   action_taken                       47252 non-null  int64  
 10  purchaser_type                     47252 non-null  int64  
 11  preapproval                        47252 non-null  int64  

### Debt to Income Ratio

The debt to income ratio has direct values, but we can specify a range instead.

In [43]:
#testvals = cfpb[len(cfpb['debt_to_income_ratio']) > 2]
# Get all possible values of debt_to_income_ratio
#testvals
def setRatioRange(data):
    firstchar = data['debt_to_income_ratio'][0]
    match firstchar:
        case '3': return '36%-<40%'
        case '4': return '40%-<50%'

dti_query = cfpb['debt_to_income_ratio'].str.len() < 3
dti_categories = cfpb.loc[dti_query].apply(lambda x: setRatioRange(x), axis=1)
cfpb.loc[dti_query, 'debt_to_income_ratio'] = dti_categories

In [44]:
cfpb['debt_to_income_ratio'].unique()

array([nan, '40%-<50%', '20%-<30%', '50%-60%', '30%-<36%', '<20%', '>60%',
       '36%-<40%'], dtype=object)

In [45]:
# save
cfpb.to_csv('../3_data_cleaned/cfpb_loan_level_LI.csv', index=False)